# 2. Cluster Analysis

Using the clusters created in program 1, identify which clusters best fit the data and areas requiring cleanup.

In [1]:
# Initialize necessary packages
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import re

# Scroll through larger dataframes (more necessary for the HDBSCAN output
from IPython.display import display, HTML

## Load Clustered Data

Begin by loading in the Excel files containing the clustered data. Save them as pandas data frames.

In [2]:
# Load KMeans Clustered Data
kmeans_19 = pd.read_excel('/Volumes/Samsung1TB/programming/data-science/baby-product-dashboard/clustered-data/kmeans.xlsx',
                          sheet_name = "cluster_19")

kmeans_42 = pd.read_excel('/Volumes/Samsung1TB/programming/data-science/baby-product-dashboard/clustered-data/kmeans.xlsx',
                          sheet_name = "cluster_42")

# Load HDBSCAN
hdb_scan = pd.read_excel('/Volumes/Samsung1TB/programming/data-science/baby-product-dashboard/clustered-data/hdbscan.xlsx',
                         sheet_name = 'umap-cosine-hdbscan')

## Identify Most Commonly Used Words by Cluster

Removing stopwords, identify the most commonly used words by each cluster. This will provide us with a better sense of the categories the clusters grouped the baby products into.

Besides just word frequency, we can also use other methods to potentially pull more meaningful words from each cluster. Because of the pre-processing in program 1 as well as teh removal of stopwords here, this may not be necessary. However, we will use this method and analyze how it performs compared to raw word frequency.

In [3]:
# Define functions to perform the above actions

# The following function removes stopwords from the analysis (also does additional processing on text, but this
# was already taken care of in the first program, so it shouldn't do anything)
def preprocess(text):
    # Confirm text is a string
    if not isinstance(text, str):
        return []
    # Lowercase and remove non-alphabetic characters
    words = re.findall(r'\b[a-z]{2,}\b', text.lower())
    # Remove stop words
    return [word for word in words if word not in ENGLISH_STOP_WORDS]

# Define function for counting the most frequently used words
def get_top_words(df, cluster_col, tokens_col, top_n=3):
    top_words = {}
    for cluster in df[cluster_col].unique():
        words = [word for tokens in df[df[cluster_col] == cluster][tokens_col] for word in tokens]
        most_common = [word for word, _ in Counter(words).most_common(top_n)]
        top_words[cluster] = most_common
    return top_words


In [4]:
# Use TF-IDF to identify more meaningful frequently used words
def get_tfidf_top_words(df, cluster_col, tokens_col, top_n=3):
    tfidf_top_words = []

    for cluster in sorted(df[cluster_col].dropna().unique()):
        # Filter rows for this cluster
        cluster_df = df[df[cluster_col] == cluster]

        # Reconstruct raw text from tokens
        cluster_docs = cluster_df[tokens_col].apply(lambda tokens: ' '.join(tokens))

        # Skip empty clusters
        if cluster_docs.empty:
            continue

        # Vectorize using TF-IDF
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(cluster_docs)
        feature_names = vectorizer.get_feature_names_out()

        # Compute average TF-IDF per word in this cluster
        tfidf_means = tfidf_matrix.mean(axis=0).A1  # convert sparse matrix to array
        top_indices = tfidf_means.argsort()[::-1][:top_n]
        top_words = [feature_names[i] for i in top_indices]

        tfidf_top_words.append({
            'cluster': cluster,
            'top_words_tfidf': top_words
        })

    return pd.DataFrame(tfidf_top_words)



### K-Means: 19

Identify the most frequently used words for the K-Means = 19 clusters dataset.

In [5]:
# Pre-process text
kmeans_19['tokens'] = kmeans_19['text'].apply(preprocess)

# Identify the most frequently used words from each cluster
top_words_dict = get_top_words(kmeans_19, 'cluster', 'tokens', top_n=3)

# Map top words back to each row
kmeans_19['top_words'] = kmeans_19['cluster'].map(top_words_dict).apply(lambda words: ', '.join(words))

In [6]:
# Print out the top 3 words for each cluster

# Keep only necessary columns
df_cluster_words_19 = kmeans_19[['cluster', 'top_words']].copy()

# Convert top words to a tuple and only get unique instances of clusters and top words
df_cluster_words_19['top_words_tuple'] = df_cluster_words_19['top_words'].apply(lambda x: tuple(x))
df_cluster_words_19 = df_cluster_words_19.drop_duplicates(subset=['cluster', 'top_words_tuple'])

# Drop the tuple column
df_cluster_words_19 = df_cluster_words_19.drop(columns='top_words_tuple')

# Sort by cluster
df_cluster_words_19 = df_cluster_words_19.sort_values(by='cluster').reset_index(drop=True)

df_cluster_words_19.head(100)

,cluster,top_words
0,0,"accessories, car, seats"
1,1,"activity, entertainment, gyms"
2,2,"blankets, swaddling, nursery"
3,3,"nursery, wall, stickers"
4,4,"care, hair, accessories"
5,5,"feeding, solid, cups"
6,6,"diaper, bags, diapering"
7,7,"diapering, diapers, cloth"
8,8,"bibs, feeding, burp"
9,9,"training, potty, pants"


Now us TF-IDF and compare results.

In [7]:
# Identify the most frequently used words from each cluster
top_words_dict_tfidf = get_tfidf_top_words(kmeans_19, 'cluster', 'tokens', top_n=3)

# SMerge tfidf top words back onto your original dataframe by cluster
kmeans_19 = kmeans_19.merge(top_words_dict_tfidf, on = 'cluster', how = 'left')

# Svae top_words_tfidf as a readable string
kmeans_19['top_words_tfidf'] = kmeans_19['top_words_tfidf'].apply(
    lambda words: ', '.join(words) if isinstance(words, list) else ''
)

In [8]:
# Print out the top 3 words for each cluster

# Keep only necessary columns
df_cluster_words_19_tfidf = kmeans_19[['cluster', 'top_words_tfidf']].copy()

# Convert top words to a tuple and only get unique instances of clusters and top words
df_cluster_words_19_tfidf['top_words_tuple'] = df_cluster_words_19_tfidf['top_words_tfidf'].apply(lambda x: tuple(x))
df_cluster_words_19_tfidf = df_cluster_words_19_tfidf.drop_duplicates(subset=['cluster', 'top_words_tuple'])

# Drop the tuple column
df_cluster_words_19_tfidf = df_cluster_words_19_tfidf.drop(columns='top_words_tuple')

# Sort by cluster
df_cluster_words_19_tfidf = df_cluster_words_19_tfidf.sort_values(by='cluster').reset_index(drop=True)

df_cluster_words_19_tfidf.head(100)

,cluster,top_words_tfidf
0,0,"accessories, seats, car"
1,1,"activity, entertainment, gyms"
2,2,"blankets, swaddling, bedding"
3,3,"wall, stickers, nursery"
4,4,"hair, care, accessories"
5,5,"feeding, solid, cups"
6,6,"diapering, bags, diaper"
7,7,"diapering, diapers, cloth"
8,8,"bibs, feeding, highchairs"
9,9,"training, potty, pants"


### K-Means: 42

Identify the most frequently used words for the K-Means = 42 clusters dataset.

In [9]:
# Pre-process text
kmeans_42['tokens'] = kmeans_42['text'].apply(preprocess)

# Identify the most frequently used words from each cluster
top_words_dict = get_top_words(kmeans_42, 'cluster', 'tokens', top_n=3)

# Map top words back to each row
kmeans_42['top_words'] = kmeans_42['cluster'].map(top_words_dict).apply(lambda words: ', '.join(words))

In [10]:
# Print out the top 3 words for each cluster

# Keep only necessary columns
df_cluster_words_42 = kmeans_42[['cluster', 'top_words']].copy()

# Convert top words to a tuple and only get unique instances of clusters and top words
df_cluster_words_42['top_words_tuple'] = df_cluster_words_42['top_words'].apply(lambda x: tuple(x))
df_cluster_words_42 = df_cluster_words_42.drop_duplicates(subset=['cluster', 'top_words_tuple'])

# Drop the tuple column
df_cluster_words_42 = df_cluster_words_42.drop(columns='top_words_tuple')

# Sort by cluster
df_cluster_words_42 = df_cluster_words_42.sort_values(by='cluster').reset_index(drop=True)

df_cluster_words_42.head(100)

,cluster,top_words
0,0,"cloth, diapers, diapering"
1,1,"storage, furniture, nursery"
2,2,"blankets, swaddling, nursery"
3,3,"feeding, solid, cups"
4,4,"pillows, nursery, maternity"
5,5,"wall, stickers, nursery"
6,6,"care, hair, accessories"
7,7,"gates, gate, safety"
8,8,"strollers, accessories, standard"
9,9,"care, health, kids"


Now use TF-IDF to compare results.

In [11]:
# Identify the most frequently used words from each cluster
top_words_dict_tfidf = get_tfidf_top_words(kmeans_42, 'cluster', 'tokens', top_n=3)

# SMerge tfidf top words back onto your original dataframe by cluster
kmeans_42 = kmeans_42.merge(top_words_dict_tfidf, on = 'cluster', how = 'left')

# Svae top_words_tfidf as a readable string
kmeans_42['top_words_tfidf'] = kmeans_42['top_words_tfidf'].apply(
    lambda words: ', '.join(words) if isinstance(words, list) else ''
)

In [12]:
# Print out the top 3 words for each cluster

# Keep only necessary columns
df_cluster_words_42_tfidf = kmeans_42[['cluster', 'top_words_tfidf']].copy()

# Convert top words to a tuple and only get unique instances of clusters and top words
df_cluster_words_42_tfidf['top_words_tuple'] = df_cluster_words_42_tfidf['top_words_tfidf'].apply(lambda x: tuple(x))
df_cluster_words_42_tfidf = df_cluster_words_42_tfidf.drop_duplicates(subset=['cluster', 'top_words_tuple'])

# Drop the tuple column
df_cluster_words_42_tfidf = df_cluster_words_42_tfidf.drop(columns='top_words_tuple')

# Sort by cluster
df_cluster_words_42_tfidf = df_cluster_words_42_tfidf.sort_values(by='cluster').reset_index(drop=True)

df_cluster_words_42_tfidf.head(100)

,cluster,top_words_tfidf
0,0,"diapers, cloth, covers"
1,1,"storage, organization, furniture"
2,2,"blankets, swaddling, bedding"
3,3,"feeding, cups, solid"
4,4,"pillows, nursery, maternity"
5,5,"nursery, stickers, wall"
6,6,"care, hair, accessories"
7,7,"gates, extensions, stair"
8,8,"strollers, accessories, standard"
9,9,"care, health, thermometers"


### HDBSCAN: 235

Identify the most frequently used words for the HDBSDCAN Procedure which produced 235 clusters.

In [13]:
# Pre-process text
hdb_scan['tokens'] = hdb_scan['text'].apply(preprocess)

# Identify the most frequently used words from each cluster
top_words_dict = get_top_words(hdb_scan, 'cluster', 'tokens', top_n=3)

# Map top words back to each row
hdb_scan['top_words'] = hdb_scan['cluster'].map(top_words_dict).apply(lambda words: ', '.join(words))

In [14]:
# Print out the top 3 words for each cluster

# Keep only necessary columns
df_cluster_words_hdbscan= hdb_scan[['cluster', 'top_words']].copy()

# Convert top words to a tuple and only get unique instances of clusters and top words
df_cluster_words_hdbscan['top_words_tuple'] = df_cluster_words_hdbscan['top_words'].apply(lambda x: tuple(x))
df_cluster_words_hdbscan = df_cluster_words_hdbscan.drop_duplicates(subset=['cluster', 'top_words_tuple'])

# Drop the tuple column
df_cluster_words_hdbscan = df_cluster_words_hdbscan.drop(columns='top_words_tuple')

# Sort by cluster
df_cluster_words_hdbscan = df_cluster_words_hdbscan.sort_values(by='cluster').reset_index(drop=True)

# Make it possible to scroll through this dataframe
# Display the DataFrame in a scrollable box
display(HTML(df_cluster_words_hdbscan.to_html(
    max_rows=300, notebook=True, index=False, justify='center',
    classes='scrollable-table', escape=False, table_id='cluster_table'
)))

# Inject some CSS to enable scrollable output
display(HTML('''
<style>
.scrollable-table {
    display: block;
    max-height: 400px;
    overflow-y: auto;
    border: 1px solid #ccc;
    padding: 10px;
}
</style>
'''))

cluster,top_words
-1,"bedding, nursery, accessories"
0,"nursery, blankets, swaddling"
1,"bedding, sheets, crib"
2,"care, hair, accessories"
3,"nursery, furniture, storage"
4,"gifts, keepsakes, hand"
5,"bedding, toddler, nursery"
6,"accessories, stroller, strollers"
7,"nursery, furniture, playards"
8,"accessories, stroller, strollers"


Now use TF-IDF.

In [15]:
# Identify the most frequently used words from each cluster
top_words_dict_tfidf = get_tfidf_top_words(hdb_scan, 'cluster', 'tokens', top_n=3)

# SMerge tfidf top words back onto your original dataframe by cluster
hdb_scan = hdb_scan.merge(top_words_dict_tfidf, on = 'cluster', how = 'left')

# Svae top_words_tfidf as a readable string
hdb_scan['top_words_tfidf'] = hdb_scan['top_words_tfidf'].apply(
    lambda words: ', '.join(words) if isinstance(words, list) else ''
)

In [16]:
# Print out the top 3 words for each cluster

# Keep only necessary columns
df_cluster_words_hdbscan_tfidf = hdb_scan[['cluster', 'top_words_tfidf']].copy()

# Convert top words to a tuple and only get unique instances of clusters and top words
df_cluster_words_hdbscan_tfidf['top_words_tuple'] = df_cluster_words_hdbscan_tfidf['top_words_tfidf'].apply(lambda x: tuple(x))
df_cluster_words_hdbscan_tfidf = df_cluster_words_hdbscan_tfidf.drop_duplicates(subset=['cluster', 'top_words_tuple'])

# Drop the tuple column
df_cluster_words_hdbscan_tfidf = df_cluster_words_hdbscan_tfidf.drop(columns='top_words_tuple')

# Sort by cluster
df_cluster_words_hdbscan_tfidf = df_cluster_words_hdbscan_tfidf.sort_values(by='cluster').reset_index(drop=True)

display(HTML(df_cluster_words_hdbscan_tfidf.to_html(
    max_rows=300, notebook=True, index=False, justify='center',
    classes='scrollable-table', escape=False, table_id='cluster_table'
)))

# Inject some CSS to enable scrollable output
display(HTML('''
<style>
.scrollable-table {
    display: block;
    max-height: 400px;
    overflow-y: auto;
    border: 1px solid #ccc;
    padding: 10px;
}
</style>
'''))

cluster,top_words_tfidf
-1,"bedding, accessories, nursery"
0,"bedding, swaddling, nursery"
1,"sheets, bedding, nursery"
2,"hair, care, clips"
3,"storage, stools, step"
4,"makers, keepsakes, hand"
5,"bedding, sets, nursery"
6,"accessories, strollers, hooks"
7,"playards, furniture, nursery"
8,"accessories, strollers, connectors"


## Identify "Outcasts"

Identify descriptions in each cluster that do not have obne of the top 3 most used words in their description.

In [17]:
# Create a function which flags records that do not have one of the top 3 words in their description
def lacks_top_words(row):
    # Ensure tokens is a list
    if not isinstance(row['tokens'], list):
        return True

    # Convert top_words and tfidf to lists if they are comma-separated strings
    top_words = row['top_words']
    tfidf_words = row['top_words_tfidf']

    if isinstance(top_words, str):
        top_words = [w.strip() for w in top_words.split(',')]
    if isinstance(tfidf_words, str):
        tfidf_words = [w.strip() for w in tfidf_words.split(',')]

    # Check if any top word or tfidf word appears in tokens
    return not any(word in row['tokens'] for word in top_words) and \
           not any(word in row['tokens'] for word in tfidf_words)


### K-Means: 19

Check the number of records which do not have a description with one of the top words from either the top words or TF-IDF top words check.

In [18]:
# Identify records in the kmeans_19 dataframe with descriptions lacking the top words of their respective cluster
# Apply the function to flag rows missing top words
kmeans_19['missing_top_word'] = kmeans_19.apply(lacks_top_words, axis=1)

In [19]:
# Get a sense of how many observations have the top words and how many do not
grouped_counts_19 = (
    kmeans_19
    .groupby(['cluster', 'top_words', 'top_words_tfidf', 'missing_top_word'])
    .size()
    .reset_index(name='count')
)

# Order by cluster
grouped_counts_19 = grouped_counts_19.sort_values(by = ['cluster', 'missing_top_word']).reset_index(drop = True)

# View the result
grouped_counts_19.head(100)


,cluster,top_words,top_words_tfidf,missing_top_word,count
0,0,"accessories, car, seats","accessories, seats, car",False,8731
1,0,"accessories, car, seats","accessories, seats, car",True,6
2,1,"activity, entertainment, gyms","activity, entertainment, gyms",False,4014
3,1,"activity, entertainment, gyms","activity, entertainment, gyms",True,9
4,2,"blankets, swaddling, nursery","blankets, swaddling, bedding",False,18377
5,2,"blankets, swaddling, nursery","blankets, swaddling, bedding",True,25
6,3,"nursery, wall, stickers","wall, stickers, nursery",False,21392
7,3,"nursery, wall, stickers","wall, stickers, nursery",True,21
8,4,"care, hair, accessories","hair, care, accessories",False,7105
9,4,"care, hair, accessories","hair, care, accessories",True,3


In [20]:
# Count the number of "True" records in this dataframe
kmeans_19.missing_top_word.value_counts()


missing_top_word
False    200665
True      17059
Name: count, dtype: int64

In [21]:
# Percentage of cases missing one of the top words
counts = kmeans_19['missing_top_word'].value_counts()
percent_true = (counts.get(True, 0) / counts.sum()) * 100
percent_false = (counts.get(False, 0) / counts.sum()) * 100
print(f"Records WITHOUT the Top Words in their Description - {percent_true:.2f}%")
print(f"Records WITH the Top Words in their Description - {percent_false:.2f}%")

Records WITHOUT the Top Words in their Description - 7.84%
Records WITH the Top Words in their Description - 92.16%


### K-Means: 42

Identify the number of records without any of the top words in their descriptions for the K-Means 42 cluster dataset.

In [22]:
# Identify records in the kmeans_42 dataframe with descriptions lacking the top words of their respective cluster
# Apply the function to flag rows missing top words
kmeans_42['missing_top_word'] = kmeans_42.apply(lacks_top_words, axis=1)

In [23]:
# Get a sense of how many observations have the top words and how many do not
grouped_counts_42 = (
    kmeans_42
    .groupby(['cluster', 'top_words', 'top_words_tfidf', 'missing_top_word'])
    .size()
    .reset_index(name='count')
)

# Order by cluster
grouped_counts_42 = grouped_counts_42.sort_values(by = ['cluster', 'missing_top_word']).reset_index(drop = True)

# View the result
display(HTML(grouped_counts_42.to_html(
    max_rows=300, notebook=True, index=False, justify='center',
    classes='scrollable-table', escape=False, table_id='cluster_table'
)))

# Inject some CSS to enable scrollable output
display(HTML('''
<style>
.scrollable-table {
    display: block;
    max-height: 400px;
    overflow-y: auto;
    border: 1px solid #ccc;
    padding: 10px;
}
</style>
'''))

cluster,top_words,top_words_tfidf,missing_top_word,count
0,"cloth, diapers, diapering","diapers, cloth, covers",False,6457
0,"cloth, diapers, diapering","diapers, cloth, covers",True,5
1,"storage, furniture, nursery","storage, organization, furniture",False,9510
1,"storage, furniture, nursery","storage, organization, furniture",True,3
2,"blankets, swaddling, nursery","blankets, swaddling, bedding",False,18371
2,"blankets, swaddling, nursery","blankets, swaddling, bedding",True,15
3,"feeding, solid, cups","feeding, cups, solid",False,9013
3,"feeding, solid, cups","feeding, cups, solid",True,1
4,"pillows, nursery, maternity","pillows, nursery, maternity",False,2687
4,"pillows, nursery, maternity","pillows, nursery, maternity",True,49


In [24]:
# Count the number of "True" records in this dataframe
kmeans_42.missing_top_word.value_counts()

missing_top_word
False    207246
True      10478
Name: count, dtype: int64

In [25]:
# Percentage of cases missing one of the top words
counts = kmeans_42['missing_top_word'].value_counts()
percent_true = (counts.get(True, 0) / counts.sum()) * 100
percent_false = (counts.get(False, 0) / counts.sum()) * 100
print(f"Records WITHOUT the Top Words in their Description - {percent_true:.2f}%")
print(f"Records WITH the Top Words in their Description - {percent_false:.2f}%")

Records WITHOUT the Top Words in their Description - 4.81%
Records WITH the Top Words in their Description - 95.19%


### HDBSCAN: 235

Finally, go ahead and perform the same analysis with the HDBSCAN Clusters.

In [26]:
# Identify records in the hdb_scan dataframe with descriptions lacking the top words of their respective cluster
# Apply the function to flag rows missing top words
hdb_scan['missing_top_word'] = hdb_scan.apply(lacks_top_words, axis=1)

In [27]:
# Get a sense of how many observations have the top words and how many do not
grouped_counts_hdbscan = (
    hdb_scan
    .groupby(['cluster', 'top_words', 'top_words_tfidf', 'missing_top_word'])
    .size()
    .reset_index(name='count')
)

# Order by cluster
grouped_counts_hdbscan = grouped_counts_hdbscan.sort_values(by = ['cluster', 'missing_top_word']).reset_index(drop = True)

# View the result
display(HTML(grouped_counts_hdbscan.to_html(
    max_rows=8000, notebook=True, index=False, justify='center',
    classes='scrollable-table', escape=False, table_id='cluster_table'
)))

# Inject some CSS to enable scrollable output
display(HTML('''
<style>
.scrollable-table {
    display: block;
    max-height: 400px;
    overflow-y: auto;
    border: 1px solid #ccc;
    padding: 10px;
}
</style>
'''))

cluster,top_words,top_words_tfidf,missing_top_word,count
-1,"bedding, nursery, accessories","bedding, accessories, nursery",False,3569
-1,"bedding, nursery, accessories","bedding, accessories, nursery",True,8433
0,"nursery, blankets, swaddling","bedding, swaddling, nursery",False,2555
1,"bedding, sheets, crib","sheets, bedding, nursery",False,3490
2,"care, hair, accessories","hair, care, clips",False,1186
3,"nursery, furniture, storage","storage, stools, step",False,997
4,"gifts, keepsakes, hand","makers, keepsakes, hand",False,405
5,"bedding, toddler, nursery","bedding, sets, nursery",False,836
6,"accessories, stroller, strollers","accessories, strollers, hooks",False,464
7,"nursery, furniture, playards","playards, furniture, nursery",False,1140


In [28]:
# Count the number of "True" records in this dataframe
hdb_scan.missing_top_word.value_counts()

missing_top_word
False    206573
True      11151
Name: count, dtype: int64

In [29]:
# Percentage of cases missing one of the top words
counts = hdb_scan['missing_top_word'].value_counts()
percent_true = (counts.get(True, 0) / counts.sum()) * 100
percent_false = (counts.get(False, 0) / counts.sum()) * 100
print(f"Records WITHOUT the Top Words in their Description - {percent_true:.2f}%")
print(f"Records WITH the Top Words in their Description - {percent_false:.2f}%")

Records WITHOUT the Top Words in their Description - 5.12%
Records WITH the Top Words in their Description - 94.88%


### Save Non-Matches
Create dataframes of just the records whose descriptions do not match the top words.

In [30]:
# Save dataframes
kmeans_19_nonmatching = kmeans_19[kmeans_19['missing_top_word'] == True]
kmeans_42_nonmatching = kmeans_42[kmeans_42['missing_top_word'] == True]
hdb_scan_nonmatching = hdb_scan[hdb_scan['missing_top_word'] == True]

## Output to Excel

Output the results produced above to Excel. Specifically, focus on the datasets with the number of top words and the breakdown of True and False by topword. Also get the datasets of records without the top words in their descriptions.

In [31]:
# K-Means 19 Datasets
with pd.ExcelWriter('/Volumes/Samsung1TB/programming/data-science/baby-product-dashboard/clustered-data/kmeans-19-breakdown.xlsx', engine='openpyxl') as writer:
    kmeans_19_nonmatching.to_excel(writer, sheet_name='cluster_19_nonmatching', index=False)
    grouped_counts_19.to_excel(writer, sheet_name='cluster_19_counts', index=False)
    df_cluster_words_19.to_excel(writer, sheet_name='cluster-by-freq')
    df_cluster_words_19_tfidf.to_excel(writer, sheet_name='cluster-tfidf')

# K-Means 42 Datasets
with pd.ExcelWriter('/Volumes/Samsung1TB/programming/data-science/baby-product-dashboard/clustered-data/kmeans-42-breakdown.xlsx', engine='openpyxl') as writer:
    kmeans_42_nonmatching.to_excel(writer, sheet_name='cluster_42_nonmatching', index=False)
    grouped_counts_42.to_excel(writer, sheet_name='cluster_42_counts', index=False)
    df_cluster_words_42.to_excel(writer, sheet_name='cluster-by-freq')
    df_cluster_words_42_tfidf.to_excel(writer, sheet_name='cluster-tfidf')

# HDBSCAN Datasets
with pd.ExcelWriter('/Volumes/Samsung1TB/programming/data-science/baby-product-dashboard/clustered-data/hdbscan-breakdown.xlsx', engine='openpyxl') as writer:
    hdb_scan_nonmatching.to_excel(writer, sheet_name='cluster_hdbscan_nonmatching', index=False)
    grouped_counts_hdbscan.to_excel(writer, sheet_name='cluster_hdbscan_counts', index=False)
    df_cluster_words_hdbscan.to_excel(writer, sheet_name='cluster-by-freq')
    df_cluster_words_hdbscan_tfidf.to_excel(writer, sheet_name='cluster-tfidf')

    

Use the above created Excel documents to perform hands-on exploratory analysis with the data. Once completed, come back to the code and identify ways to break down the clusters in more detail.

## Further Cluster Analysis

After exploring the data independently, it seems like we will want to explore the HDBSCAN Cluster results. While we will attempt to shrink the number of categories accordingly, the results from K-Means = 19 do not appear as useful, due to the poor performance of Cluster 8, which captures a large amount of categories ranging from strollers, to bibs, to high chairs, etc.

K-Means 42 also appears to have its difficulties. Cluster 9 also appears to havepoor performance with a large number of records no matching any of the top words. With that said, HDBSCAN contains way too many clusters to use in a dashboard, so the goal will be to ultimately reduce the number of clusters overall. We could use the K-Means 42 results, but HDBSCAN proves to work better with BERT models and may still not provide the proper distinction between clusters that HDBSCAN does (may be easier to work down from too many clusters as opposed to beginning with a smaller amount).

We will start by looking at the HDBSCAN results, see how many meaningful clusters we cna reduce it to by grouping several clusters together.

### HDBSCAN Breakdown - Reduce into Meaningful Clusters

In [32]:
# Create a dataset with just the top words (both tf-idf and just by freq)
hdb_cluster_additional_analysis = df_cluster_words_hdbscan.merge(df_cluster_words_hdbscan_tfidf, left_on='cluster', right_on='cluster')

# Combine all top words together into one string
hdb_cluster_additional_analysis['top_words'] = hdb_cluster_additional_analysis['top_words'].apply(
    lambda x: ' '.join(x.split(', '))
)

hdb_cluster_additional_analysis['top_words_tfidf'] = hdb_cluster_additional_analysis['top_words_tfidf'].apply(
    lambda x: ' '.join(x.split(', '))
)

hdb_cluster_additional_analysis['top_words_combined'] = (
    hdb_cluster_additional_analysis['top_words'] + ' ' + hdb_cluster_additional_analysis['top_words_tfidf']
)

# Vectorize the cluster labels
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(hdb_cluster_additional_analysis['top_words_combined'])

# Measure the similarity across clusters
similarity_matrix = cosine_similarity(X)

# Group similar clusters
n_final_groups = 60  # tweak this
cluster_grouper = AgglomerativeClustering(n_clusters=n_final_groups, metric='precomputed', linkage='average')
group_assignments = cluster_grouper.fit_predict(1 - similarity_matrix)

hdb_cluster_additional_analysis['merged_cluster'] = group_assignments

# View the result
display(HTML(hdb_cluster_additional_analysis.to_html(
    max_rows=300, notebook=True, index=False, justify='center',
    classes='scrollable-table', escape=False, table_id='cluster_table'
)))

# Inject some CSS to enable scrollable output
display(HTML('''
<style>
.scrollable-table {
    display: block;
    max-height: 400px;
    overflow-y: auto;
    border: 1px solid #ccc;
    padding: 10px;
}
</style>
'''))

cluster,top_words,top_words_tfidf,top_words_combined,merged_cluster
-1,bedding nursery accessories,bedding accessories nursery,bedding nursery accessories bedding accessories...,4
0,nursery blankets swaddling,bedding swaddling nursery,nursery blankets swaddling bedding swaddling nu...,4
1,bedding sheets crib,sheets bedding nursery,bedding sheets crib sheets bedding nursery,4
2,care hair accessories,hair care clips,care hair accessories hair care clips,55
3,nursery furniture storage,storage stools step,nursery furniture storage storage stools step,0
4,gifts keepsakes hand,makers keepsakes hand,gifts keepsakes hand makers keepsakes hand,5
5,bedding toddler nursery,bedding sets nursery,bedding toddler nursery bedding sets nursery,4
6,accessories stroller strollers,accessories strollers hooks,accessories stroller strollers accessories stro...,9
7,nursery furniture playards,playards furniture nursery,nursery furniture playards playards furniture n...,10
8,accessories stroller strollers,accessories strollers connectors,accessories stroller strollers accessories stro...,9


In [33]:
# Explore some of these merged clusters
hdb_cluster_additional_analysis[hdb_cluster_additional_analysis['merged_cluster'] == 58]

,cluster,top_words,top_words_tfidf,top_words_combined,merged_cluster
12,11,cloth diapering diapers,accessories covers diapering,cloth diapering diapers accessories covers dia...,58
45,44,diapering disposable diapers,diapers disposable diapering,diapering disposable diapers diapers disposabl...,58
87,86,cloth diapers diapering,diapers cloth diapering,cloth diapers diapering diapers cloth diapering,58
94,93,cloth inserts diapers,accessories diapering diapers,cloth inserts diapers accessories diapering di...,58
125,124,wet cloth diapers,accessories diapering diapers,wet cloth diapers accessories diapering diapers,58
190,189,cloth covers diapering,diapering diapers covers,cloth covers diapering diapering diapers covers,58
227,226,diaper cloth diapers,diaper cloth diapers,diaper cloth diapers diaper cloth diapers,58


In [36]:
# Explore some of these merged clusters
hdb_cluster_additional_analysis[hdb_cluster_additional_analysis['merged_cluster'] == 59]

,cluster,top_words,top_words_tfidf,top_words_combined,merged_cluster,top_words_combined_list
169,168,safety electrical proofing,safety electrical shock,safety electrical proofing safety electrical s...,59,"[safety, electrical, proofing, safety, electri..."


In [ ]:
# Create a list of all top words associated with each merged_cluster

# Convert the combined words string to a list to make it easier to iterate over
hdb_cluster_additional_analysis['top_words_combined_list'] = (
    hdb_cluster_additional_analysis['top_words_combined'].apply(lambda x: x.split())
)

# Define a function to identify all unique words in each list
def get_all_unique_words(word_lists):
    all_words = [word for word_list in word_lists for word in word_list]
    return sorted(set(all_words))  # Optional: sort alphabetically

# Group by merged_cluster and compute both word lists and original cluster values
merged_cluster_summary = (
    hdb_cluster_additional_analysis
    .groupby('merged_cluster')
    .agg({
        'top_words_combined_list': get_all_unique_words,
        'cluster': lambda x: sorted(set(x))  # original cluster IDs
    })
    .reset_index()
)

# Turn lists into readable strings
merged_cluster_summary['unique_top_words'] = merged_cluster_summary['top_words_combined_list'].apply(lambda x: ', '.join(x))
merged_cluster_summary['original_clusters'] = merged_cluster_summary['cluster'].apply(lambda x: ', '.join(map(str, x)))

# Drop the raw list columns if you only want the final strings
merged_cluster_summary = merged_cluster_summary[['merged_cluster', 'original_clusters', 'unique_top_words']]

merged_cluster_summary.head(1000)

,merged_cluster,original_clusters,unique_top_words
0,0,"3, 12, 34, 68, 106, 110, 153, 154, 174, 190, 230","basket, bins, chests, feeding, food, furniture..."
1,1,"10, 98, 123, 144, 149, 157, 185","accessories, bandana, carriers, cart, covers, ..."
2,2,"36, 51, 53, 193, 214, 221","bath, bathing, bubble, care, feeding, hooded, ..."
3,3,"9, 43, 61, 75, 130, 211","accessories, backpacks, bags, caddies, cloth, ..."
4,4,"-1, 0, 1, 5, 25, 40, 42, 56, 59, 63, 91, 104, ...","accessories, bags, bedding, blankets, crib, gu..."
5,5,"4, 55, 64, 96, 135, 213","baskets, boxes, furniture, gift, gifts, hand, ..."
6,6,"137, 142, 172, 181, 199, 215, 217","beverages, bottle, curtain, feeding, foods, fo..."
7,7,"20, 52, 89, 92, 129, 132, 148, 204","accessories, breast, breastfeeding, complete, ..."
8,8,"196, 234","boards, book, memory, nursery, wall"
9,9,"6, 8, 21, 30, 31, 49, 57, 81, 97, 115, 118, 15...","accessories, bags, bunting, connectors, cup, h..."


In [54]:
# Merge merged_cluster and top_words info back onto the original HDBSCAN dataset
df_hdbscan_final = hdb_scan.merge(
    hdb_cluster_additional_analysis[['cluster', 'merged_cluster', 'top_words_combined']],
    on='cluster',
    how='left'
)

df_hdbscan_final.head(100)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,...,category_select,categories_clean,text,cluster,tokens,top_words,top_words_tfidf,missing_top_word,merged_cluster,top_words_combined
0,Baby,"Chicco Viaro Travel System, Teak",4.6,125,['Aluminum' 'Imported'\n 'Convenient one-hand ...,['Product Description'\n 'For ultimate conveni...,NaN,{'hi_res': array(['https://m.media-amazon.com/...,"{'title': array(['Viaro Demo Video', 'Chicco V...",Chicco,...,['Baby Products' 'Strollers & Accessories' 'St...,strollers & accessories strollers travel systems,strollers & accessories strollers travel systems,219,"[strollers, accessories, strollers, travel, sy...","strollers, accessories, travel","strollers, travel, systems",False,9,strollers accessories travel strollers travel ...
1,AMAZON FASHION,Kisbaby Four Layers Muslin Lightweight Unisex ...,5.0,2,"['95% Cotton, 4 Layer Muslin, Hand Wash in Col...",['You can choose bigger size If you confuse ab...,NaN,{'hi_res': array(['https://m.media-amazon.com/...,"{'title': array([], dtype=object), 'url': arra...",Kisbaby,...,['Baby Products' 'Nursery' 'Bedding' 'Blankets...,nursery bedding blankets & swaddling wearable ...,nursery bedding blankets & swaddling wearable ...,91,"[nursery, bedding, blankets, swaddling, wearab...","blankets, nursery, bedding","blankets, wearable, swaddling",False,4,blankets nursery bedding blankets wearable swa...
2,Baby,EZTOTZ Meals with Milton - USA Made Toddler & ...,4.4,37,['WHAT IS MILTON?: Milton is the fun way for y...,[],22.99,{'hi_res': array(['https://m.media-amazon.com/...,"{'title': array([], dtype=object), 'url': arra...",EZTOTZ,...,EZTOTZ Meals with Milton - USA Made Toddler & ...,eztotz meals with milton - usa made toddler & ...,eztotz meals with milton - usa made toddler & ...,231,"[eztotz, meals, milton, usa, toddler, feeding,...","silicone, breast, set","bottle, set, silicone",False,21,silicone breast set bottle set silicone
3,Baby,Nuby iMonster Toddler Bowl,4.4,52,['Makes feeding fun for baby and easier for pa...,['When babies begin to show interest in feedin...,NaN,{'hi_res': array(['https://m.media-amazon.com/...,"{'title': array([], dtype=object), 'url': arra...",Nuby,...,['Baby Products' 'Feeding' 'Solid Feeding' 'Di...,feeding solid feeding dishes,feeding solid feeding dishes,47,"[feeding, solid, feeding, dishes]","feeding, solid, dishes","feeding, solid, dishes",False,39,feeding solid dishes feeding solid dishes
4,Amazon Home,mDesign Slim Storage Organizer Container Bin w...,4.5,235,['SMART STORAGE: This large capacity slim bin ...,"[""The mDesign clear storage bins for kids supp...",NaN,{'hi_res': array(['https://m.media-amazon.com/...,"{'title': array([], dtype=object), 'url': arra...",mDesign,...,['Baby Products' 'Nursery' 'Furniture' 'Storag...,nursery furniture storage & organization stora...,nursery furniture storage & organization stora...,34,"[nursery, furniture, storage, organization, st...","storage, nursery, furniture","storage, bins, organization",False,0,storage nursery furniture storage bins organiz...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Baby,"Luxe Basics So Softy Fitted Crib Sheet, Lavender",4.5,6,['100% Polyester' 'Made in US'\n 'Fitted crib ...,['For sweet dreams and sound sleep look no fur...,NaN,{'hi_res': array(['https://m.media-amazon.com/...,"{'title': array([], dtype=object), 'url': arra...",Luxe Basics,...,['Baby Products' 'Nursery' 'Bedding' 'Baby Bed...,nursery bedding bedding sheets crib sheets,nursery bedding bedding sheets crib sheets,1,"[nursery, bedding, bedding, sheets, crib, sheets]","bedding, sheets, crib","sheets, bedding, nursery",False,4,bedding sheets crib sheets bedding nursery
96,Baby,HIPHOP PANDA Baby Waterproof Crib Mattress Pro...,4.8,7,['Front is 100% Bamboo - Back is 100% Waterpro...,['🍃【COZY BAMBOO FIBER】 HIPHOP PANDA mattress p...,19.99,{'hi_res': array(['https://m.media-amazon.com/...,{'title': array

In [50]:
# Save the following results to Excel
with pd.ExcelWriter('/Volumes/Samsung1TB/programming/data-science/baby-product-dashboard/clustered-data/hdbscan-grouped-clusters.xlsx', engine='openpyxl') as writer:
    merged_cluster_summary.to_excel(writer, sheet_name='clusters-simplified', index=False)
    df_hdbscan_final.to_excel(writer, sheet_name='updated-hdbscan-dataset', index=False)

## Final Groupings

Now that we have analyzed the different clusters and grouped them together in order to reduce the number of overall clusters, let's combine this information to the main dataset. After doing this, we will go ahead and split out the dataset into two groups - one classified due to have a desciption in the top words for the cluster, and those which do not. 

Note that we will also remove the -1 cluster observations since these are cases that could not be adequately grouped.

In [75]:
# Use the function from earlier to flag records with results without matching top words
#df_hdbscan_final['missing_top_word'] = df_hdbscan_final.apply(lacks_top_words, axis=1)

baby_products_categorized_merged_cluster = df_hdbscan_final[(df_hdbscan_final['missing_top_word'] == False) & \
                                                            (df_hdbscan_final['cluster'] != -1)]

categorized_count = len(baby_products_categorized_merged_cluster.parent_asin.value_counts())

print(f"There are {categorized_count:,} observations with categorized values (top words in string and not cluster -1).")

require_categorization_after_merged_clusters = df_hdbscan_final[(df_hdbscan_final['missing_top_word'] == True) | \
                                                            (df_hdbscan_final['cluster'] == -1)]

uncategorized_count = len(require_categorization_after_merged_clusters.parent_asin.value_counts())

print(f"There are {uncategorized_count:,} observations with categorized values (top words in string and not cluster -1).")


There are 203,004 observations with categorized values (top words in string and not cluster -1).
There are 14,720 observations with categorized values (top words in string and not cluster -1).


In [76]:
# Output these results to Excel for use in the next program
with pd.ExcelWriter('/Volumes/Samsung1TB/programming/data-science/baby-product-dashboard/categorized-data/hdbscan-merged-cluster-categorized.xlsx', engine='openpyxl') as writer:
    baby_products_categorized_merged_cluster.to_excel(writer, sheet_name='categorized', index=False)
    require_categorization_after_merged_clusters.to_excel(writer, sheet_name='NOT-categorized', index=False)